### **Imports**

In [13]:
from os.path import join, realpath, dirname, exists, abspath, isfile, isdir
from os import mkdir as mk, name as os_name, getcwd, environ, pathsep, rename, listdir
from wget import download
from uuid import uuid1
from time import sleep
import tensorflow as tf
import cv2

### **Definitions**

In [8]:
# Settings & Options 🔧
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
PRETRAINED_MODEL_ZIP_NAME = f'{PRETRAINED_MODEL_NAME}.tar.gz'
API_MODELS_URL = 'https://github.com/tensorflow/models'
PROTOC_VERSION = '3.15.6'
PROTOC_URL = f'https://github.com/protocolbuffers/protobuf/releases/download/v{PROTOC_VERSION}/protoc-{PROTOC_VERSION}-win64.zip'
PROTOC_ZIP_NAME = f'protoc-{PROTOC_VERSION}-win64.zip'
MODEL_NAME = 'model-v1' # current model name being used within 'workspace/models' directory
LATEST_CHECKPOINT = 11
BATCH_SIZE = 8
TRAINING_STEPS = 10_000

# Paths 📂
ROOT_DIR = getcwd()
API_MODELS_DIR = join(ROOT_DIR, 'models')
API_RESEARCH_DIR = join(API_MODELS_DIR, 'research')
API_OBJECT_DETECTION_DIR = join(API_RESEARCH_DIR, 'object_detection')
API_SETUP_SCRIPT_PATH = join(API_OBJECT_DETECTION_DIR, 'packages', 'tf2', 'setup.py')
API_VERIFICATION_SCRIPT_PATH = join(API_OBJECT_DETECTION_DIR, 'builders', 'model_builder_tf2_test.py')
API_SLIM_DIR = join(API_RESEARCH_DIR, 'slim')
PROTOC_DIR = join(ROOT_DIR, 'protoc')
PROTOC_BIN_DIR = join(PROTOC_DIR, 'bin')
TRAIN_SCRIPT_PATH = join(API_MODELS_DIR, 'research', 'object_detection', 'model_main_tf2.py')
WORKSPACE_DIR = join(ROOT_DIR, 'workspace')
MODELS_DIR = join(WORKSPACE_DIR, 'models')
PRETRAINED_MODELS_DIR = join(WORKSPACE_DIR, 'pre-trained-models')
SCRIPTS_DIR = join(ROOT_DIR, 'scripts')
TF_RECORD_SCRIPT_PATH = join(SCRIPTS_DIR, 'generate_tfrecord.py')
LABELMAP_SCRIPT_PATH = join(SCRIPTS_DIR, 'generate_labelmap.py')
IMAGES_DIR = join(WORKSPACE_DIR, 'images')
TEST_IMAGES_DIR = join(IMAGES_DIR, 'test')
TRAIN_IMAGES_DIR = join(IMAGES_DIR, 'train')
VALIDATION_IMAGES_DIR = join(IMAGES_DIR, 'validation')
LABELING_IMAGES_DIR = join(IMAGES_DIR, 'labeling')
COLLECTED_IMAGES_DIR = join(LABELING_IMAGES_DIR, 'collected')
CAPTURED_IMAGES_DIR = join(IMAGES_DIR, 'captured')
ANNOTATIONS_DIR = join(WORKSPACE_DIR, 'annotations')
LABELMAP_PATH = join(ANNOTATIONS_DIR, 'label_map.pbtxt')
TRAIN_TF_RECORD_PATH = join(ANNOTATIONS_DIR, 'train.record')
TEST_TF_RECORD_PATH = join(ANNOTATIONS_DIR, 'test.record')
MODEL_DIR = join(MODELS_DIR, MODEL_NAME)
PRETRAINED_MODEL_DIR = join(PRETRAINED_MODELS_DIR, PRETRAINED_MODEL_NAME)
PRETRAINED_CONFIG_PATH = join(PRETRAINED_MODEL_DIR, 'pipeline.config')
PRETRAINED_CHECKPOINT_PATH = join(PRETRAINED_MODEL_DIR, 'checkpoint', 'ckpt-0')
MODEL_CONFIG_PATH = join(MODEL_DIR, 'pipeline.config')
CURRENT_CP_PATH = join(MODEL_DIR, f'ckpt-{LATEST_CHECKPOINT}')

# Constants 🧮
SIGNS = [
  # letters
  'a', 'b', 'c', 'd',
  
  # words
  'llamar', 'caminar', 'comunicar', 'ladron'
  
  # phrases
  
]

# Util 🆘
def mkdir(path: str, debug=False):
  if not exists(path):
    mk(path)
  else:
    if debug:
      print(f'{path} already exists!')

def r(path: str):
  return path.replace('\\', '/')

### **Capturing**

In [ ]:
# Options
SNAPSHOTS = 10
WAIT_PER_SIGN = 5
WAIT_PER_SNAPSHOT = 2

In [ ]:
# Capturing all signs!
cap = cv2.VideoCapture(0) # NOTE: change '0' if camera device is not being picked up

for sign in SINGS:
  print(f'Capturing sign {sign} in {WAIT_PER_SIGN}s')
  sleep(WAIT_PER_SIGN)

  for i in range(SNAPSHOTS):
    print(f'Collecting image {i + 1} for sign {sign} in {WAIT_PER_SNAPSHOT}')
    sleep(WAIT_PER_SNAPSHOT)
    
    ret, frame = cap.read()
    img_path = join(CAPTURED_IMAGES_DIR, f'{sign}.{uuid1()}.jpg')
    cv2.imwrite(img_path, frame)
    cv2.imshow('frame', frame)
    
cap.release()
cv2.destroyAllWindows()

In [ ]:
# Capturing single sign!
cap = cv2.VideoCapture(0) # NOTE: change '0' if camera device is not being picked up
sign = 'a'


print(f'Capturing sign {sign} in {WAIT_PER_SIGN}s')
sleep(WAIT_PER_SIGN)

for i in range(SNAPSHOTS):
  print(f'Collecting image {i + 1} for sign {sign} in {WAIT_PER_SNAPSHOT}')
  sleep(WAIT_PER_SNAPSHOT)
  
  ret, frame = cap.read()
  img_path = join(CAPTURED_IMAGES_DIR, f'{sign}.{uuid1()}.jpg')
  cv2.imwrite(img_path, frame)
  cv2.imshow('frame', frame)
    
cap.release()
cv2.destroyAllWindows()

### **Raw Image Preping**
> *Renames and prepares images for labeling*

In [15]:
# SINGLE FOLDER
_SIGN = 'a'
SIGN_DIR = join(LABELING_IMAGES_DIR, _SIGN)

if not exists(SIGN_DIR): raise Exception(f'make sure {SIGN_DIR} exists!')

for img_name in listdir(SIGN_DIR):
  src_img_path = join(SIGN_DIR, img_name)
  if isfile(src_img_path):
    dest_img_path = join(COLLECTED_IMAGES_DIR, f'{_SIGN}.{uuid1()}.jpg')
    rename(src_img_path, dest_img_path)


In [ ]:
# ALL FOLDERS
for _SIGN in listdir(LABELING_IMAGES_DIR):
  if _SIGN == 'collected': continue
  
  SIGN_DIR = join(LABELING_IMAGES_DIR, _SIGN)
  if not exists(SIGN_DIR): raise Exception(f'make sure {SIGN_DIR} exists!')

  for img_name in listdir(SIGN_DIR):
    src_img_path = join(SIGN_DIR, img_name)
    if isfile(src_img_path):
      dest_img_path = join(COLLECTED_IMAGES_DIR, f'{_SIGN}.{uuid1()}.jpg')
      rename(src_img_path, dest_img_path)

### **Labeling**

In [ ]:
!labelimg

### **Object Detection API Installation**

In [ ]:
# TODO: TEST!
if os_name == 'posix':
    print('using posix')
    !apt-get install protobuf-compiler
    !cd {API_OBJECT_DETECTION_DIR} && protoc protos/*.proto --python_out=. && cp {API_SETUP_SCRIPT_PATH} . && python -m pip install . 

if os_name == 'nt':
    print('using nt')
    download(PROTOC_URL)
    !move {PROTOC_ZIP_NAME} {PROTOC_DIR}
    !cd {PROTOC_DIR} && tar -xf {PROTOC_ZIP_NAME}
    environ['PATH'] += pathsep + abspath(PROTOC_BIN_DIR)
    RL_SCRIPT_PATH = join('object_detection', 'packages', 'tf2', 'setup.py')
    !cd {API_RESEARCH_DIR} && protoc object_detection/protos/*.proto --python_out=. && copy {RL_SCRIPT_PATH} setup.py
    !cd {API_RESEARCH_DIR} && python setup.py build && python setup.py install
    !cd {API_SLIM_DIR} && pip install -e . 

In [ ]:
# Verify Installation
!python {API_VERIFICATION_SCRIPT_PATH}

### **Object Detection API Imports**

> **note**: *must run [Object Detection API installation](#Object_Detection_API_Installation) section and restart kernel for object_detection module to work!*

In [7]:
from google.protobuf import text_format
from object_detection.protos import pipeline_pb2
from object_detection.utils import config_util as cu
from object_detection.utils import label_map_util as lu
from object_detection.builders import model_builder as mb
from object_detection.utils import visualization_utils as vu

### **Setup pre-trained Model**

In [ ]:
# TODO: TEST!
if os_name =='posix':
  print('using posix')
  !wget {PRETRAINED_MODEL_URL}
  !mv {PRETRAINED_MODEL_NAME + '.tar.gz'} {PRETRAINED_MODELS_DIR}
  !cd {PRETRAINED_MODELS_DIR} && tar -zxvf {PRETRAINED_MODEL_NAME + '.tar.gz'}

if os_name == 'nt':
  print('using nt')
  download(PRETRAINED_MODEL_URL)
  !move { PRETRAINED_MODEL_NAME + '.tar.gz' } {PRETRAINED_MODELS_DIR}
  !cd {PRETRAINED_MODELS_DIR} && tar -zxvf {PRETRAINED_MODEL_NAME + '.tar.gz'}

##### **Set up model**

In [ ]:
mkdir(MODEL_DIR)

if os_name =='posix':
  print('using posix')
  !cp {PRETRAINED_CONFIG_PATH} {MODEL_DIR}

if os_name == 'nt':
  print('using nt')
  !copy {PRETRAINED_CONFIG_PATH} {MODEL_DIR}

config = pipeline_pb2.TrainEvalPipelineConfig()

with tf.io.gfile.GFile(MODEL_CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
  proto_str = f.read()                                                                                                                                                                                                                                          
  text_format.Merge(proto_str, config)
    
config.model.ssd.num_classes = len(SIGNS)
config.train_config.batch_size = BATCH_SIZE
config.train_config.fine_tune_checkpoint = PRETRAINED_CHECKPOINT_PATH
config.train_config.fine_tune_checkpoint_type = "detection"
config.train_config.use_bfloat16 = False
config.train_input_reader.label_map_path= LABELMAP_PATH
config.train_input_reader.tf_record_input_reader.input_path[:] = [TRAIN_TF_RECORD_PATH]
config.eval_input_reader[0].label_map_path = LABELMAP_PATH
config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [TEST_TF_RECORD_PATH]

config_text = text_format.MessageToString(config)
with tf.io.gfile.GFile(MODEL_CONFIG_PATH, "wb") as f:
  f.write(config_text)

### **Prep-train**
> **note**: *make sure you've labeled and partition your dataset*

##### **Generating Label Map**

In [6]:
with open(LABELMAP_PATH, 'w') as f:
  id = 0
  for sign in SIGNS:
    id += 1
    f.write('item { \n')
    f.write(f'\tname: \'{sign}\'\n')
    f.write(f'\tid: {id}\n')
    f.write('}\n')

##### **Generating TF Records**

In [ ]:
!python {TF_RECORD_SCRIPT_PATH} -x {TEST_IMAGES_DIR} -l {LABELMAP_PATH} -o {TEST_TF_RECORD_PATH} 
!python {TF_RECORD_SCRIPT_PATH} -x {TRAIN_IMAGES_DIR} -l {LABELMAP_PATH} -o {TRAIN_TF_RECORD_PATH} 

### **Training**

In [ ]:
# from cmd line
POSIX_SHELL = True
tensorboard_cmd = f'tensorboard --logdir={MODEL_DIR}'
train_cmd = f'python {TRAIN_SCRIPT_PATH} --model_dir={MODEL_DIR} --pipeline_config_path={MODEL_CONFIG_PATH} --num_train_steps={TRAINING_STEPS}'
print('run the following command(s):\n')
print(r(tensorboard_cmd) if POSIX_SHELL else tensorboard_cmd)
print('\n')
print(r(train_cmd) if POSIX_SHELL else train_cmd)

In [ ]:
# from jupyter (NOT RECOMMENDED)
!tensorboard --logdir={MODEL_DIR}
!python {TRAIN_SCRIPT_PATH} --model_dir={MODEL_DIR} --pipeline_config_path={MODEL_CONFIG_PATH} --num_train_steps={TRAINING_STEPS}

> **note**: *its going to take a while depending on the* `BATCH_SIZE`, `TRAINNING_STEPS` *and whether you're using a GPU or not*

### **Detecting**

In [9]:
import numpy as np

# Load pipeline config and build a detection model
configs = cu.get_configs_from_pipeline_file(MODEL_CONFIG_PATH)
detection_model = mb.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(CURRENT_CP_PATH).expect_partial()

@tf.function
def detect_fn(image):
  image, shapes = detection_model.preprocess(image)
  prediction_dict = detection_model.predict(image, shapes)
  detections = detection_model.postprocess(prediction_dict, shapes)
  return detections

category_index = lu.create_category_index_from_labelmap(LABELMAP_PATH)

# Setup capture
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while True: 
  ret, frame = cap.read()
  image_np = np.array(frame)
  
  input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
  detections = detect_fn(input_tensor)
  
  num_detections = int(detections.pop('num_detections'))
  detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
  detections['num_detections'] = num_detections

  # detection_classes should be ints.
  detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

  label_id_offset = 1
  image_np_with_detections = image_np.copy()

  vu.visualize_boxes_and_labels_on_image_array(
    image_np_with_detections,
    detections['detection_boxes'],
    detections['detection_classes']+label_id_offset,
    detections['detection_scores'],
    category_index,
    use_normalized_coordinates=True,
    max_boxes_to_draw=5,
    min_score_thresh=.75,
    agnostic_mode=False
  )

  cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
  
  if cv2.waitKey(1) & 0xFF == ord('q'):
    cap.release()
    break